This experiment is done on the ground truth. The ground truth is created based on the NMVW constituents' display names and the rdfs:labels retrieved from the wikidata. It only contains constituesnts where the wikidata identifies is known.

In [ ]:
import sys
sys.path.append("..") 

In [ ]:
import pickle, pandas
from data_matching import exact_string_match, match_lastName, match_with_abbreviation, match_fuzzy_string
from calculate_result import calculate_result

In [ ]:
with open('data/ground_truth.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
candidates = set([str(item) for _, row in data.iterrows() for item in row['wiki_label']])

In [ ]:

print(f"The size of ground truth: {len(data.index)} and size of candidates: {len(candidates)}")

# Exact string matching

In [ ]:
# Exact string matching
result_exact = exact_string_match(data, candidates)
exact_match = len(result_exact.loc[result_exact["match"] == "YES"])
print(f"The number of positive match is {exact_match}")

In [ ]:
with open("results/NMVWtoWikiExactMatchResults.pkl", "wb") as handle:
    pickle.dump(result_exact, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df = pandas.read_pickle("results/NMVWtoWikiExactMatchResults.pkl")
print(len(df.index))

In [ ]:
calculate_result("results/NMVWtoWikiExactMatchResults.pkl")

# Surname Matching

In [ ]:
result_surname = match_lastName(data, candidates)
surname_match = len(result_surname.loc[result_surname["match"] == "YES"])
print(f"The number of positive match is {surname_match}")

In [ ]:
with open("results/NMVWtoWikiSurnameMatchResults.pkl", "wb") as handle:
    pickle.dump(result_surname, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df = pandas.read_pickle("results/NMVWtoWikiSurnameMatchResults.pkl")
print(len(df.index))

In [ ]:
calculate_result("results/NMVWtoWikiSurnameMatchResults.pkl")

# Abbreviation Matching

In [ ]:
result_abbreviation.loc[result_abbreviation["match"] == "YES"].to_csv("temp.csv")

In [ ]:
result_abbreviation = match_with_abbreviation(data, candidates)
abbreviationlen_match = len(result_abbreviation.loc[result_abbreviation["match"] == "YES"])
print(f"The number of positive match is {abbreviationlen_match}")

In [ ]:
with open("results/NMVWtoWikiAbbreviationMatchResults.pkl", "wb") as handle:
    pickle.dump(result_abbreviation, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df = pandas.read_pickle("results/NMVWtoWikiAbbreviationMatchResults.pkl")
print(len(df.index))

In [ ]:
calculate_result("results/NMVWtoWikiAbbreviationMatchResults.pkl")

# Fuzzy String Match

In [ ]:
result_fuzzymatch = match_fuzzy_string(data, candidates, max_score=75)
fuzzy_match = len(result_fuzzymatch.loc[result_fuzzymatch["match"] == "YES"])
print(f"The number of positive match is {fuzzy_match}")

In [ ]:
with open("results/NMVWtoWikiFuzzyStringMatchResults.pkl", "wb") as handle:
    pickle.dump(result_fuzzymatch, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df = pandas.read_pickle("results/NMVWtoWikiFuzzyStringMatchResults.pkl")
print(len(df.index))

In [ ]:
calculate_result("results/NMVWtoWikiFuzzyStringMatchResults.pkl")